In [ ]:
import os # os module upload
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from PIL import Image
import cv2

0차 *필터링*

*   CYMK, RGBA를 따로 분리



In [ ]:
def three_channel_filtering(image):
    if len(image.shape) == 3:
        if image.shape[2] == 3:
            return True
        else:
            return False
    else:
        return False

first filtering by the comparison of pixels

In [ ]:
def filtering(image):
    count = 0
    image_x = image.shape[1] - 1
    image_y = image.shape[0] - 1

    pixel_list = [image[(0,0)], image[(image_y, 0)], image[(0, image_x)], image[(image_y, image_x)]]

    com_list = list(combinations(pixel_list, 2))

    for com in com_list:
        if all(com[0] == com[1]):
            count += 1
    
    if count >= 3:
        return True
    else:
        return False

1st filtering process

In [ ]:
org_path = ('/content/drive/MyDrive/Data/org')
pre_path = ('/content/drive/MyDrive/Data/preprocessing_1')
final_pre_path = ('/content/drive/MyDrive/Data/preprocessing_final')
filtered_path = ('/content/drive/MyDrive/Data/1st_filtering')

dir_paths = os.listdir(org_path)

count = 0

for dir_path in dir_paths:
    org_paths = os.path.join(org_path, dir_path)
    pre_paths = os.path.join(pre_path, dir_path)
    final_pre_paths = os.path.join(final_pre_path, dir_path)
    filtered_paths = os.path.join(filtered_path, dir_path)
    
    # os.mkdir(pre_paths)
    # os.mkdir(final_pre_paths)
    # os.mkdir(filtered_paths)

    image_paths = os.listdir(org_paths)

    for image_path in image_paths:
        count += 1
        org_image_path = os.path.join(org_paths, image_path)
        pre_image_path = os.path.join(pre_paths, image_path)
        final_pre_image_path = os.path.join(final_pre_paths, image_path)
        filtered_image_path = os.path.join(filtered_paths, image_path)

        image = load_image(org_image_path)
        print(org_image_path, count)
        if not three_channel_filtering(image):
            shutil.move(org_image_path, final_pre_image_path)
        elif filtering(image):
            shutil.move(org_image_path, pre_image_path)
        else:
            shutil.move(org_image_path, filtered_image_path)

2nd filtering (cropped data)

In [ ]:
def cropping(image, image_gray):
    threshold1 = 30
    threshold2 = 60

    image_canny = cv2.Canny(image_gray, threshold1, threshold2)

    contour_list = np.argwhere(image_canny>0)
    y1,x1 = contour_list.min(axis=0)
    y2,x2 = contour_list.max(axis=0)

    return image[y1:y2, x1:x2]

이미지의 네 모서리 픽셀값이 같은지를 비교하여 실사 filtering

In [ ]:
def filtering2(image):
    image_x = image.shape[1] - 1
    image_y = image.shape[0] - 1

    pixel_list = [image[(0,0)], image[(image_y, 0)], image[(0, image_x)], image[(image_y, image_x)]]

    com_list = list(combinations(pixel_list, 2))

    for com in com_list:
        if all(com[0] == com[1]):
            return True
    
    return False

2nd filtering process 

In [ ]:
org_path = ('/content/drive/MyDrive/Data/preprocessing_1')
pre_path = ('/content/drive/MyDrive/Data/preprocessing_2')
filtered_path = ('/content/drive/MyDrive/Data/2nd_filtering')

dir_paths = os.listdir(org_path)

count = 0

for dir_path in dir_paths:
    org_paths = os.path.join(org_path, dir_path)
    pre_paths = os.path.join(pre_path, dir_path)
    filtered_paths = os.path.join(filtered_path, dir_path)
    
    # os.mkdir(pre_paths)
    # os.mkdir(filtered_paths)

    image_paths = os.listdir(org_paths)

    for image_path in image_paths:
        count += 1
        org_image_path = os.path.join(org_paths, image_path)
        pre_image_path = os.path.join(pre_paths, image_path)
        filtered_image_path = os.path.join(filtered_paths, image_path)

        image = load_image(org_image_path)
        image_gray = load_gray_image(org_image_path)
        print(org_image_path, count)

        crop_image = cropping(image, image_gray)

        if filtering2(crop_image):
            shutil.move(org_image_path, pre_image_path)
        else:
            shutil.move(org_image_path, filtered_image_path)

**after 2 steps filtering** Datasets upload

In [ ]:
image_path = '/content/drive/MyDrive/Data/preprocessing_2/'

label_path = os.listdir(image_path) # listdir에 파일의 경로가 들어있다

label_paths = [os.path.join(image_path,label_path[i]) for i in range(len(label_path))] # L2_#까지 포함된 경로

image_jpg = [os.listdir(label_paths[i]) for i in range(len(label_paths))] # JPG명

image_paths = [] # 여기에 최종 경로가 저장됨
for i in range(len(label_paths)):
    sub_paths = []
    for j in range(len(image_jpg[i])):
        sub_paths.append(os.path.join(label_paths[i], image_jpg[i][j]))
    image_paths.append(sub_paths)

처음으로 실사로 거른 셋에 대한 경로 업로드

In [ ]:
image_path_1 = '/content/drive/MyDrive/Data/1st_filtering/'

label_path_1 = os.listdir(image_path_1) # listdir에 파일의 경로가 들어있다

label_paths_1 = [os.path.join(image_path_1,label_path_1[i]) for i in range(len(label_path_1))]

image_jpg_1 = [os.listdir(label_paths_1[i]) for i in range(len(label_paths_1))]

image_paths_1 = []
for i in range(len(label_paths_1)):
    sub_paths = []
    for j in range(len(image_jpg_1[i])):
        sub_paths.append(os.path.join(label_paths_1[i], image_jpg_1[i][j]))
    image_paths_1.append(sub_paths)

특정 label에 특히 1차로 거른 실사들이 많다.

*   우선 이 클래스들의 사진들의 픽셀값을 0 ~ 50, 50 ~ 200까지는 10단위로, 200 ~ 255까지로 분리한 것을 dataframe으로 만들어줌
*   맨 뒤에 해당 사진의 픽셀 분포도의 standard deviation을 구하여 dataframe의 열로 추가
*   해당 dataframe이 ISOLATION FOREST의 이상치 탐지의 학습 데이터가 될 것이다. 









In [ ]:
dist_real = []
img_idx = []
pic_class = ["L2_3", "L2_12", "L2_24", "L2_41", "L2_50"]
for i in range(len(label_path)):
    if label_path[i] in pic_class:
        img_idx.append(i)
for i in img_idx:
    dist_sub = []
    print(i)
    for img in image_paths_1[i]:
        img = Image.open(img)
        img = np.array(img)
        check = img.flatten()
        dis = []
        cnt = 0
        for j in range(50, 210, 10):
            if j == 50:
                dis.append(len(check[check <= j]))
                cnt += 1
            else:
                dis.append(len(check[check <= j]) - sum(dis[:cnt]))
                cnt += 1
                if j == 200:
                    dis.append(len(check[check <= 255]) - sum(dis[:cnt+1]))
        dist_sub.append(dis)
    dist_real.append(dist_sub)

# IsoForest에 돌릴 실사 데이터 모음. 약 1900개
dist_df = pd.concat([pd.DataFrame(dist_real[0]),pd.DataFrame(dist_real[1]),pd.DataFrame(dist_real[2]),pd.DataFrame(dist_real[3]),pd.DataFrame(dist_real[4])], ignore_index = True)
d_real = dist_df.copy()
std_set = []
for i in range(len(d_real)):
    std_set.append(np.std(d_real.iloc[i]))
d_real["std"] = std_set

2차까지 일러스트라고 분류된 사진 역시 픽셀 범위 별로 DataFrame을 만든다
이는 IsolationForest의 테스트 데이터가 된다.

In [ ]:
dist_picture = []
for i in range(len(image_paths)):
    sub_picture = []
    print(i)
    for img in image_paths[i]:
        img = Image.open(img)
        img = np.array(img)
        check = img.flatten()
        dis = []
        cnt = 0
        for j in range(50, 210, 10):
            if j == 50:
                dis.append(len(check[check <= j]))
                cnt += 1
            else:
                dis.append(len(check[check <= j]) - sum(dis[:cnt]))
                cnt += 1
                if j == 200:
                    dis.append(len(check[check <= 255]) - sum(dis[:cnt+1]))
        sub_picture.append(dis)
    dist_picture.append(sub_picture)

IsolationForest: anomaly detection에 활용
hyperparameter: n_estimators = 100, contamination = 0.115, random_state = 42
사진 데이터 1906개에 대해서 학습 진행 -> 후에 이미지 데이터로 predict

In [ ]:
from sklearn.ensemble import IsolationForest
iso = IsolationForest(n_estimators = 100, contamination = 0.115, random_state = 42)
iso.fit(d_real)

IsolationForest(contamination=0.115, random_state=42)

In [ ]:
# 3차의 저장소 경로 지정
image_path_picture = '/content/drive/MyDrive/Data/3rd_filtering/picture'
image_path_real = '/content/drive/MyDrive/Data/3rd_filtering/real'

# label_path = os.listdir(image_path) # listdir에 파일의 경로가 들어있다

# label_paths = [os.path.join(image_path,label_path[i]) for i in range(len(label_path))] # L2_#까지 포함된 경로

# image_jpg = [os.listdir(label_paths[i]) for i in range(len(label_paths))] # JPG명

In [ ]:
for i in range(len(dist_picture)):
    d_or = pd.DataFrame(dist_picture[i])
    print(i)
    std_set = []
    for j in range(len(d_or)):
        std_set.append(np.std(d_or.iloc[j]))
    d_or["std"] = std_set
    ans = iso.predict(d_or)
    print(len(ans[ans==1]))
    idx = []
    for k in range(len(ans)):
        # 이미지 데이터는 -1로서 판단
        if ans[k] == -1:
            pic_route = os.path.join(image_path_picture, label_path[i])
            path = os.path.join(pic_route, image_jpg[i][k])
            img = Image.open(image_paths[i][k])
            img = np.array(img)
            cv_image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            cv2.imwrite(path, cv_image)
        else:
            real_route = os.path.join(image_path_real, label_path[i])
            path = os.path.join(real_route, image_jpg[i][k])
            img = Image.open(image_paths[i][k])
            img = np.array(img)
            cv_image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            cv2.imwrite(path, cv_image)


이미지 업로드

In [ ]:
image_path_final = '/content/drive/MyDrive/Data/3rd_filtering/real'

label_path_final = os.listdir(image_path_final) # listdir에 파일의 경로가 들어있다

label_paths_final = [os.path.join(image_path_final,label_path_final[i]) for i in range(len(label_path_final))]

image_jpg_final = [os.listdir(label_paths_final[i]) for i in range(len(label_paths_final))]

image_paths_final = []
for i in range(len(label_paths_final)):
    sub_paths = []
    for j in range(len(image_jpg_final[i])):
        sub_paths.append(os.path.join(label_paths_final[i], image_jpg_final[i][j]))
    image_paths_final.append(sub_paths)